<a href="https://colab.research.google.com/github/Aditya-Chopra30/hydroguard/blob/main/ML_Model___Hydroguard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install scikit-learn folium pandas

# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import folium
from sklearn.metrics import mean_squared_error
from IPython.display import IFrame

# Generate the dataset (as previously created)
np.random.seed(42)
data_size = 1000
locations = [f"Location_{i}" for i in range(1, 101)]
data = {
    "Location": np.random.choice(locations, data_size),
    "RoadCondition": np.random.choice(["Clear", "Moderate Traffic", "Heavy Traffic"], data_size),
    "DistanceToFire (km)": np.round(np.random.uniform(0.5, 20, data_size), 2),
    "EstimatedTravelTime (min)": np.round(np.random.uniform(5, 60, data_size), 2),
    "WaterResourceAvailable (liters)": np.random.choice([1000, 2000, 5000, 10000], data_size),
    "RefillStationDistance (km)": np.round(np.random.uniform(1, 10, data_size), 2),
    "PriorityLevel": np.random.choice(["High", "Medium", "Low"], data_size),
}
df = pd.DataFrame(data)

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=["Location", "RoadCondition", "PriorityLevel"], drop_first=True)

# Features and target variables
X = df_encoded.drop(["DistanceToFire (km)", "WaterResourceAvailable (liters)"], axis=1)
y_distance = df_encoded["DistanceToFire (km)"]
y_water = df_encoded["WaterResourceAvailable (liters)"]

# Split the data into training and test sets
X_train, X_test, y_train_dist, y_test_dist, y_train_water, y_test_water = train_test_split(
    X, y_distance, y_water, test_size=0.2, random_state=42
)

# Train Random Forest Regressor for Distance
model_distance = RandomForestRegressor(n_estimators=100, random_state=42)
model_distance.fit(X_train, y_train_dist)

# Train Random Forest Regressor for Water Availability
model_water = RandomForestRegressor(n_estimators=100, random_state=42)
model_water.fit(X_train, y_train_water)

# Evaluate the models
dist_preds = model_distance.predict(X_test)
water_preds = model_water.predict(X_test)

print(f"Distance RMSE: {np.sqrt(mean_squared_error(y_test_dist, dist_preds))}")
print(f"Water Availability RMSE: {np.sqrt(mean_squared_error(y_test_water, water_preds))}")

# User input for location and time
location_input = input("Enter the location (e.g., Location_1 to Location_100): ")
estimated_travel_time = float(input("Enter the estimated travel time (in minutes): "))

# Simulate user input as a row in the dataset
# Since we're using encoded variables, we need to match the location and time into the encoded dataframe structure

# Create a new row for the input
new_input = df_encoded.iloc[0:1].copy()
new_input.loc[:, :] = 0  # Set everything to 0

# Set the correct location
location_column = f"Location_{location_input.split('_')[1]}"
if location_column in new_input.columns:
    new_input[location_column] = 1

# Set the travel time
new_input["EstimatedTravelTime (min)"] = estimated_travel_time

# Predict shortest distance and water availability
predicted_distance = model_distance.predict(new_input)
predicted_water = model_water.predict(new_input)

print(f"\nPredicted Distance to Fire: {predicted_distance[0]:.2f} km")
print(f"Predicted Water Resource Available: {predicted_water[0]:.2f} liters")

# Map visualization using Folium
# Assuming random lat/lon coordinates for locations for demonstration
location_coordinates = {
    f"Location_{i}": [28.60 + np.random.uniform(-0.05, 0.05), 77.20 + np.random.uniform(-0.05, 0.05)] for i in range(1, 101)
}

# Get coordinates of the fire location (based on input)
fire_coords = location_coordinates[location_input]

# Initialize the map centered around the fire location
map = folium.Map(location=fire_coords, zoom_start=13)

# Add a marker for the fire location
folium.Marker(location=fire_coords, popup="Fire Location", icon=folium.Icon(color="red")).add_to(map)

# Add markers for nearby refill stations (simulate nearby refill stations)
for i in range(3):  # 3 nearby stations
    refill_station_coords = [
        fire_coords[0] + np.random.uniform(-0.02, 0.02),
        fire_coords[1] + np.random.uniform(-0.02, 0.02)
    ]
    folium.Marker(location=refill_station_coords, popup=f"Refill Station {i+1}", icon=folium.Icon(color="blue")).add_to(map)
m
# Save and display the map
map.save("fire_brigade_map.html")
IFrame("fire_brigade_map.html", width=700, height=500)

Distance RMSE: 5.5989432949218205
Water Availability RMSE: 3787.825761568238
Enter the location (e.g., Location_1 to Location_100): Location_5
Enter the estimated travel time (in minutes): 5


<ipython-input-2-96f956c88b89>:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  new_input.loc[:, :] = 0  # Set everything to 0
<ipython-input-2-96f956c88b89>:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  new_input.loc[:, :] = 0  # Set everything to 0
<ipython-input-2-96f956c88b89>:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  new_input.loc[:, :] = 0  # Set everything to 0
<ipython-input-2-96f956c88b89>:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a futur

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- DistanceToFire (km)
- WaterResourceAvailable (liters)


In [1]:
# Install required libraries
!pip install scikit-learn folium pandas gradio googlemaps openweathermapy

# Import required libraries
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import folium
from sklearn.metrics import mean_squared_error
from IPython.display import IFrame
import time
from openweathermapy.core import get_current

# Generate the dataset (as previously created)
np.random.seed(42)
data_size = 1000
locations = [f"Location_{i}" for i in range(1, 101)]
data = {
    "Location": np.random.choice(locations, data_size),
    "RoadCondition": np.random.choice(["Clear", "Moderate Traffic", "Heavy Traffic"], data_size),
    "DistanceToFire (km)": np.round(np.random.uniform(0.5, 20, data_size), 2),
    "EstimatedTravelTime (min)": np.round(np.random.uniform(5, 60, data_size), 2),
    "WaterResourceAvailable (liters)": np.random.choice([1000, 2000, 5000, 10000], data_size),
    "RefillStationDistance (km)": np.round(np.random.uniform(1, 10, data_size), 2),
    "PriorityLevel": np.random.choice(["High", "Medium", "Low"], data_size),
}
df = pd.DataFrame(data)

# Simulated traffic API, weather API, and dynamic water resource allocation
def simulate_traffic(location):
    traffic_conditions = {
        "Clear": 10,
        "Moderate Traffic": 30,
        "Heavy Traffic": 60
    }
    road_condition = np.random.choice(["Clear", "Moderate Traffic", "Heavy Traffic"])
    return traffic_conditions[road_condition], road_condition

def simulate_weather(location):
    # Simulate weather data (replace with actual OpenWeatherMap API)
    weather_conditions = ["Sunny", "Rainy", "Windy", "Stormy"]
    temperature = np.random.uniform(20, 40)
    wind_speed = np.random.uniform(0, 15)
    return np.random.choice(weather_conditions), temperature, wind_speed

def update_water_resources(location, water_used):
    water_resources = np.random.choice([5000, 10000, 2000, 8000], 1)[0]
    water_resources -= water_used
    return max(water_resources, 0)

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=["Location", "RoadCondition", "PriorityLevel"], drop_first=True)

# Features and target variables
X = df_encoded.drop(["DistanceToFire (km)", "WaterResourceAvailable (liters)"], axis=1)
y_distance = df_encoded["DistanceToFire (km)"]
y_water = df_encoded["WaterResourceAvailable (liters)"]

# Split the data into training and test sets
X_train, X_test, y_train_dist, y_test_dist, y_train_water, y_test_water = train_test_split(
    X, y_distance, y_water, test_size=0.2, random_state=42
)

# Train Random Forest Regressor for Distance
model_distance = RandomForestRegressor(n_estimators=100, random_state=42)
model_distance.fit(X_train, y_train_dist)

# Train Random Forest Regressor for Water Availability
model_water = RandomForestRegressor(n_estimators=100, random_state=42)
model_water.fit(X_train, y_train_water)

# Define function to handle user inputs and predictions
def predict(location_input):
    # Simulate traffic, weather, and water resource allocation
    traffic_time, road_condition = simulate_traffic(location_input)
    weather, temperature, wind_speed = simulate_weather(location_input)
    water_used = np.random.randint(500, 1500)
    remaining_water = update_water_resources(location_input, water_used)

    # Simulate user input as a row in the dataset
    new_input = pd.DataFrame(np.zeros((1, len(X.columns))), columns=X.columns)

    # Set the correct location
    location_column = f"Location_{location_input.split('_')[1]}"
    if location_column in new_input.columns:
        new_input[location_column] = 1

    # Predict shortest distance and water availability
    predicted_distance = model_distance.predict(new_input)
    predicted_water = model_water.predict(new_input)

    # Map visualization using Folium
    # Assuming random lat/lon coordinates for locations for demonstration
    location_coordinates = {
        f"Location_{i}": [28.60 + np.random.uniform(-0.05, 0.05), 77.20 + np.random.uniform(-0.05, 0.05)] for i in range(1, 101)
    }

    # Get coordinates of the fire location (based on input)
    fire_coords = location_coordinates[location_input]

    # Initialize the map centered around the fire location
    map = folium.Map(location=fire_coords, zoom_start=13)

    # Add a marker for the fire location
    folium.Marker(location=fire_coords, popup="Fire Location", icon=folium.Icon(color="red")).add_to(map)

    # Add markers for nearby refill stations (simulate nearby refill stations)
    for i in range(3):  # 3 nearby stations
        refill_station_coords = [
            fire_coords[0] + np.random.uniform(-0.02, 0.02),
            fire_coords[1] + np.random.uniform(-0.02, 0.02)
        ]
        folium.Marker(location=refill_station_coords, popup=f"Refill Station {i+1}", icon=folium.Icon(color="blue")).add_to(map)

    # Save and display the interactive map using IFrame
    map.save("fire_brigade_map.html")

    # Return predictions and map
    return f"Predicted Distance to Fire: {predicted_distance[0]:.2f} km\n" \
           f"Predicted Water Resource Available: {predicted_water[0]:.2f} liters\n" \
           f"Remaining Water After Use: {remaining_water:.2f} liters\n" \
           f"Traffic Condition: {road_condition}, Estimated Time: {traffic_time} minutes\n" \
           f"Weather: {weather}, Temperature: {temperature}°C, Wind Speed: {wind_speed} m/s", "fire_brigade_map.html"

# Create Gradio UI with updated API
iface = gr.Interface(
    fn=predict,
    inputs=gr.components.Textbox(label="Enter the location (e.g., Location_1 to Location_100)"),
    outputs=[gr.components.Textbox(label="Prediction"), gr.components.File(label="Map")],
    title="Fire Brigade System",
    description="Enter the location of the fire to get predictions on distance, water resources, traffic, and weather.",
)

# Launch the Gradio interface
iface.launch(share=True)

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/13

In [2]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import folium

# Data generation
np.random.seed(42)
data_size = 1000
locations = [f"Location_{i}" for i in range(1, 101)]
data = {
    "Location": np.random.choice(locations, data_size),
    "RoadCondition": np.random.choice(["Clear", "Moderate Traffic", "Heavy Traffic"], data_size),
    "DistanceToFire (km)": np.round(np.random.uniform(0.5, 20, data_size), 2),
    "EstimatedTravelTime (min)": np.round(np.random.uniform(5, 60, data_size), 2),
    "WaterResourceAvailable (liters)": np.random.choice([1000, 2000, 5000, 10000], data_size),
    "RefillStationDistance (km)": np.round(np.random.uniform(1, 10, data_size), 2),
    "PriorityLevel": np.random.choice(["High", "Medium", "Low"], data_size),
}
df = pd.DataFrame(data)

# Simulate traffic, weather, and water resource updates
def simulate_traffic(location):
    traffic_conditions = {
        "Clear": 10,
        "Moderate Traffic": 30,
        "Heavy Traffic": 60
    }
    road_condition = np.random.choice(["Clear", "Moderate Traffic", "Heavy Traffic"])
    return traffic_conditions[road_condition], road_condition

def simulate_weather(location):
    weather_conditions = ["Sunny", "Rainy", "Windy", "Stormy"]
    temperature = np.random.uniform(20, 40)
    wind_speed = np.random.uniform(0, 15)
    return np.random.choice(weather_conditions), temperature, wind_speed

def update_water_resources(location, water_used):
    water_resources = np.random.choice([5000, 10000, 2000, 8000], 1)[0]
    water_resources -= water_used
    return max(water_resources, 0)

# Encode Data
df_encoded = pd.get_dummies(df, columns=["Location", "RoadCondition", "PriorityLevel"], drop_first=True)

X = df_encoded.drop(["DistanceToFire (km)", "WaterResourceAvailable (liters)"], axis=1)
y_distance = df_encoded["DistanceToFire (km)"]
y_water = df_encoded["WaterResourceAvailable (liters)"]

X_train, X_test, y_train_dist, y_test_dist, y_train_water, y_test_water = train_test_split(
    X, y_distance, y_water, test_size=0.2, random_state=42
)

# Train models
model_distance = RandomForestRegressor(n_estimators=100, random_state=42)
model_distance.fit(X_train, y_train_dist)

model_water = RandomForestRegressor(n_estimators=100, random_state=42)
model_water.fit(X_train, y_train_water)

# Define the prediction function
def predict(location_input):
    traffic_time, road_condition = simulate_traffic(location_input)
    weather, temperature, wind_speed = simulate_weather(location_input)
    water_used = np.random.randint(500, 1500)
    remaining_water = update_water_resources(location_input, water_used)

    new_input = pd.DataFrame(np.zeros((1, len(X.columns))), columns=X.columns)
    location_column = f"Location_{location_input.split('_')[1]}"
    if location_column in new_input.columns:
        new_input[location_column] = 1

    predicted_distance = model_distance.predict(new_input)
    predicted_water = model_water.predict(new_input)

    location_coordinates = {
        f"Location_{i}": [28.60 + np.random.uniform(-0.05, 0.05), 77.20 + np.random.uniform(-0.05, 0.05)] for i in range(1, 101)
    }

    fire_coords = location_coordinates[location_input]

    map = folium.Map(location=fire_coords, zoom_start=13)

    folium.Marker(location=fire_coords, popup="Fire Location", icon=folium.Icon(color="red")).add_to(map)

    for i in range(3):
        refill_station_coords = [
            fire_coords[0] + np.random.uniform(-0.02, 0.02),
            fire_coords[1] + np.random.uniform(-0.02, 0.02)
        ]
        folium.Marker(location=refill_station_coords, popup=f"Refill Station {i+1}", icon=folium.Icon(color="blue")).add_to(map)

    map.save("fire_brigade_map.html")

    return f"Predicted Distance to Fire: {predicted_distance[0]:.2f} km\n" \
           f"Predicted Water Resource Available: {predicted_water[0]:.2f} liters\n" \
           f"Remaining Water After Use: {remaining_water:.2f} liters\n" \
           f"Traffic Condition: {road_condition}, Estimated Time: {traffic_time} minutes\n" \
           f"Weather: {weather}, Temperature: {temperature}°C, Wind Speed: {wind_speed} m/s", "fire_brigade_map.html"

# Gradio interface with enhanced UI/UX
iface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(label="Enter the location (e.g., Location_1 to Location_100)", placeholder="e.g. Location_42"),
    outputs=[gr.Textbox(label="Prediction"), gr.File(label="Map")],
    title="🚒 Fire Brigade Prediction System 🚒",
    description="""
        🔥 Enter the fire location to get a predictive analysis on distance, water resources, traffic, and weather conditions.
        Firefighters can respond more efficiently with real-time predictions!
    """,
    theme="default",
    allow_flagging="never",
    examples=["Location_1", "Location_42", "Location_78"],
    css="""
        body {
            background-color: #121212;
            color: #E0E0E0;
            font-family: 'Montserrat', sans-serif;
        }
        h1 {
            color: #FF3E30;
            text-align: center;
            font-size: 2.5em;
            margin-top: 20px;
        }
        .gradio-input {
            margin-bottom: 20px;
        }
        .gradio-btn-primary {
            background: linear-gradient(to right, #FF3E30, #FF7F50);
            border: none;
            color: white;
            padding: 12px;
            border-radius: 8px;
            cursor: pointer;
        }
        .gradio-btn-primary:hover {
            background: linear-gradient(to right, #FF7F50, #FF3E30);
        }
        .gradio-output {
            background: #1E1E1E;
            padding: 10px;
            border-radius: 10px;
            font-size: 1.1em;
        }
    """
)

# Launch the interface
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7f256392caa2089101.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# Install required libraries
!pip install scikit-learn folium pandas

# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import folium
from sklearn.metrics import mean_squared_error
from IPython.display import IFrame

# Generate the dataset (as previously created)
np.random.seed(42)
data_size = 1000
locations = [f"Location_{i}" for i in range(1, 101)]
data = {
    "Location": np.random.choice(locations, data_size),
    "RoadCondition": np.random.choice(["Clear", "Moderate Traffic", "Heavy Traffic"], data_size),
    "DistanceToFire (km)": np.round(np.random.uniform(0.5, 20, data_size), 2),
    "EstimatedTravelTime (min)": np.round(np.random.uniform(5, 60, data_size), 2),
    "WaterResourceAvailable (liters)": np.random.choice([1000, 2000, 5000, 10000], data_size),
    "RefillStationDistance (km)": np.round(np.random.uniform(1, 10, data_size), 2),
    "PriorityLevel": np.random.choice(["High", "Medium", "Low"], data_size),
}
df = pd.DataFrame(data)

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=["Location", "RoadCondition", "PriorityLevel"], drop_first=True)

# Features and target variables
X = df_encoded.drop(["DistanceToFire (km)", "WaterResourceAvailable (liters)"], axis=1)
y_distance = df_encoded["DistanceToFire (km)"]
y_water = df_encoded["WaterResourceAvailable (liters)"]

# Split the data into training and test sets
X_train, X_test, y_train_dist, y_test_dist, y_train_water, y_test_water = train_test_split(
    X, y_distance, y_water, test_size=0.2, random_state=42
)

# Train Random Forest Regressor for Distance
model_distance = RandomForestRegressor(n_estimators=100, random_state=42)
model_distance.fit(X_train, y_train_dist)

# Train Random Forest Regressor for Water Availability
model_water = RandomForestRegressor(n_estimators=100, random_state=42)
model_water.fit(X_train, y_train_water)

# Evaluate the models
dist_preds = model_distance.predict(X_test)
water_preds = model_water.predict(X_test)

print(f"Distance RMSE: {np.sqrt(mean_squared_error(y_test_dist, dist_preds))}")
print(f"Water Availability RMSE: {np.sqrt(mean_squared_error(y_test_water, water_preds))}")

# User input for location only
location_input = input("Enter the location (e.g., Location_1 to Location_100): ")

# Simulate user input as a row in the dataset
# Since we're using encoded variables, we need to match the location into the encoded dataframe structure

# Create a new row for the input
new_input = df_encoded.iloc[0:1].copy()
new_input.loc[:, :] = 0  # Set everything to 0

# Set the correct location
location_column = f"Location_{location_input.split('_')[1]}"
if location_column in new_input.columns:
    new_input[location_column] = 1

# Predict shortest distance and water availability
predicted_distance = model_distance.predict(new_input)
predicted_water = model_water.predict(new_input)

print(f"\nPredicted Distance to Fire: {predicted_distance[0]:.2f} km")
print(f"Predicted Water Resource Available: {predicted_water[0]:.2f} liters")

# Map visualization using Folium
# Assuming random lat/lon coordinates for locations for demonstration
location_coordinates = {
    f"Location_{i}": [28.60 + np.random.uniform(-0.05, 0.05), 77.20 + np.random.uniform(-0.05, 0.05)] for i in range(1, 101)
}

# Get coordinates of the fire location (based on input)
fire_coords = location_coordinates[location_input]

# Initialize the map centered around the fire location
map = folium.Map(location=fire_coords, zoom_start=13)

# Add a marker for the fire location
folium.Marker(location=fire_coords, popup="Fire Location", icon=folium.Icon(color="red")).add_to(map)

# Add markers for nearby refill stations (simulate nearby refill stations)
for i in range(3):  # 3 nearby stations
    refill_station_coords = [
        fire_coords[0] + np.random.uniform(-0.02, 0.02),
        fire_coords[1] + np.random.uniform(-0.02, 0.02)
    ]
    folium.Marker(location=refill_station_coords, popup=f"Refill Station {i+1}", icon=folium.Icon(color="blue")).add_to(map)

# Save and display the map
map.save("fire_brigade_map.html")
IFrame("fire_brigade_map.html", width=700, height=500)